In [5]:
import pandas as pd
import numpy as np
import scipy as sp
import miRNET, miRNET_enrichment
Targets = miRNET.Targets(path_to_miRTarBase='./baseData/hsa_miRTarBase.csv')
import warnings
import collections
from matplotlib import pyplot as plt
import random
from scipy.stats import chi2_contingency
import json
import networkx as nx

In [6]:
miRnames_up = ['miR-29a-3p', 'miR-93-5p', 'miR-133a-3p', 'miR-199a-3p', 
               'miR-221-3p', 'miR-222-3p', 'miR-451a', 'miR-497-5p', 'miR-21-5p', 
               'miR-1-3p', 'miR-19b-3p', 'miR-155-5p']

In [7]:
def mir_list_to_target_list(miR_names):
    colnames = ['miR', 'n_targets', 'n_heart_targets', 'n_LCC', 'n_key_genes']
    table = pd.DataFrame(columns=colnames)
    miR_key_nodes = dict()
    for miR in miR_names:
        print('miR-name:', miR, sep=' ')
        miR_targets = Targets.get_targets(miR)
        if miR_targets == 1:
            warnings.warn('use another name')
            continue
        tis_gene_set = miRNET.tissue_selector(ans=0, tissue_id=23)
        MirNet = miRNET.MainNet() #Load String db and create gene-gene interaction network
        MirNet.get_LCC() #get the largest connected component from the network
        MirNet.select_nodes(miR_targets) # select the part of LCC containing only the miRNA target genes
        MirNet.select_nodes(tis_gene_set) # select the part of LCC containing only the tissue target genes
        kne = miRNET.KeyNodesExtractor(MirNet) # creating an object for extracting key nodes and storing their meta information
        miR_key_nodes[miR] = list(kne.extraction()) #key nodes extraction
        
        print()
        
        nx.to_pandas_edgelist(MirNet.LCC).to_csv('../miRNAs_LCC/edge_list_' + miR + '.csv', index=False, index_label=False)
        row = [miR]
        row.append(len(miR_targets))
        row.append(len(tis_gene_set.intersection(miR_targets)))
        row.append(len(MirNet.LCC.nodes()))
        row.append(len(miR_key_nodes[miR]))
        
        row = pd.DataFrame(row).transpose()
        row.columns = colnames

        table = pd.concat([table, row], axis=0)
    return [miR_key_nodes, table]

In [8]:
up_miRs = mir_list_to_target_list(miRnames_up)

miR-name: miR-29a-3p
I found a miRNA with name: hsa-miR-29a-3p
and  265 unique targets
Gene universe is...
heart muscle was used
your tissue is  heart muscle  number of genes:  7944

miR-name: miR-93-5p
I found a miRNA with name: hsa-miR-93-5p
and  1220 unique targets
Gene universe is...
heart muscle was used
your tissue is  heart muscle  number of genes:  7944

miR-name: miR-133a-3p
I found a miRNA with name: hsa-miR-133a-3p
and  131 unique targets
Gene universe is...
heart muscle was used
your tissue is  heart muscle  number of genes:  7944

miR-name: miR-199a-3p
I found a miRNA with name: hsa-miR-199a-3p
and  114 unique targets
Gene universe is...
heart muscle was used
your tissue is  heart muscle  number of genes:  7944

miR-name: miR-221-3p
I found a miRNA with name: hsa-miR-221-3p
and  368 unique targets
Gene universe is...
heart muscle was used
your tissue is  heart muscle  number of genes:  7944

miR-name: miR-222-3p
I found a miRNA with name: hsa-miR-222-3p
and  394 unique tar

In [10]:
up_miRs[1].to_csv('../table_1.csv', sep=';')

In [ ]:
up_dict = up_miRs[0]

In [ ]:
with open('../miRNAs_key_genes/up_miRNAs_key_genes_dict.json', 'w') as outfile:
    json.dump(up_dict, outfile)

In [ ]:
def get_df_edges(enrich_res, miRname):
    pathes = list(enrich_res.react_dict.keys())
    miR_col = [miRname for i in range(len(pathes))]
    return pd.DataFrame(list(zip(miR_col, pathes)), columns=['miR', 'path'])

In [ ]:
#list_paths = list()
data_edges = pd.DataFrame(columns=['miR', 'path'])
for miR in up_dict.keys():
    enrich_res = miRNET_enrichment.reactome_enrichment(up_dict[miR], species='Homo sapiens')
    enrich_res = miRNET_enrichment.reac_pars(enrich_res)
#    list_paths = list_paths + list(enrich_res.react_dict.keys())
    data_edges = pd.concat([data_edges, get_df_edges(enrich_res, miR)], axis=0)

In [ ]:
data_edges.to_csv('../miRNAs_pathes_bigraphs/up_bigraph_edge.csv', index=False, index_label=False, sep=';')